IMPORTAZIONE LIBRERIE

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import glob
import array 
import scipy.signal as signal
#Processing libraries
from sklearn.preprocessing import StandardScaler,MinMaxScaler
import seaborn as sns
from scipy.stats import zscore
from sklearn.multiclass import OneVsRestClassifier

# Model libraries
from sklearn.linear_model import LogisticRegression, SGDClassifier, Perceptron
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE

# Testing libraries
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV, learning_curve
from sklearn.metrics import accuracy_score, log_loss, confusion_matrix, classification_report, roc_curve, auc

# remove warnings
import warnings
warnings.filterwarnings('ignore')

CAMBIO PATH PER ANALISI RISPETTO AL LOG DI VOLO CONSIDERAT

In [2]:
path_file = os.path.abspath(os.getcwd())
print(path_file)
os.chdir(path_file)

c:\Users\chiar\OneDrive\Desktop\MantenzionePreventiva\ProgettoManutenzionePreventiva


ARRAY CHE CONTERRA' I DATAFRAME DI CIASCUN VOLO (PER UN TOTALE DI 18)

In [3]:
data_path = os.path.join(path_file,'*.csv') #lista di tutti gli elementi di estensione csv nella cartella path
csv_list = glob.glob(data_path) #converte data path in un output Unix-like (ls) (*.csv -> lista di elementi con estensione csv)
dataframe_collection = []
df_1 = pd.DataFrame()

# ciclo per scorrere tutti i csv
for csv_file in csv_list: # ciclo che scorre i csv nella cartella path
    df = pd.read_csv(csv_file) 
    #df = df.to_numpy()
    #df = df.to_records(index=False)
    #dataframe_collection.append(df)
    df_1 = df_1.append(df,ignore_index=True)
    #print(df)


In [ ]:
#df_1.to_csv('totale.csv', index=False)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
#display(df_1)

In [ ]:
df_1 = df_1.sample(frac=1).reset_index(drop=True)
print(df_1)

In [4]:
df_1 = df_1.drop(['tempo'], axis=1)
display(df_1)

,<>(GyrX),var(GyrX),kurt(GyrX),RMS(GyrX),freq1(GyrX),freq2(GyrX),amp1(GyrX),amp2(GyrX),<>(GyrY),var(GyrY),...,amp2(C13),<>(C14),var(C14),kurt(C14),RMS(C14),freq1(C14),freq2(C14),amp1(C14),amp2(C14),Guasto
0,-0.032100,0.017081,-0.399284,0.134398,78.0,96.0,3.596543,3.414090,-0.014483,0.028038,...,1316.635778,1530.205128,1397.957802,-0.375963,1530.660547,116.0,115.0,1356.236798,982.813328,2
1,-0.032346,0.017054,-0.400010,0.134356,102.0,80.0,3.601798,3.359201,-0.014131,0.028001,...,1347.166450,1530.380682,1404.823329,-0.386721,1530.838288,113.0,112.0,1351.732999,889.971776,2
2,-0.032590,0.017026,-0.400594,0.134314,98.0,79.0,3.785400,3.155365,-0.013642,0.028006,...,1335.348799,1530.555241,1411.588559,-0.397326,1531.015002,116.0,115.0,1367.356776,931.422173,2
3,-0.032832,0.016999,-0.401113,0.134271,102.0,71.0,3.964024,3.257887,-0.013156,0.028010,...,1322.356341,1530.728814,1418.254861,-0.407778,1531.190697,111.0,110.0,1437.986779,1042.276447,2
4,-0.033265,0.017017,-0.412325,0.134447,102.0,87.0,4.069144,2.898697,-0.013003,0.027940,...,1308.221788,1530.901408,1424.823586,-0.418075,1531.365382,118.0,116.0,1502.075347,1117.254428,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1344,-0.048472,0.006254,-0.082119,0.092673,0.0,119.0,13.705592,4.427470,0.036188,0.004072,...,721.379609,1549.740920,5073.978831,0.974985,1551.373138,133.0,131.0,1511.376776,995.696782,0
1345,-0.048289,0.006253,-0.081993,0.092571,0.0,118.0,13.703035,4.344276,0.036631,0.004143,...,730.061710,1549.922705,5075.374156,0.980104,1551.555190,134.0,132.0,1514.809977,861.450797,0
1346,-0.048212,0.006240,-0.075446,0.092463,0.0,119.0,13.655833,4.213446,0.037145,0.004243,...,694.279610,1550.103614,5076.696968,0.985247,1551.736343,141.0,139.0,1518.825061,730.400470,0
1347,-0.048179,0.006225,-0.068067,0.092367,0.0,130.0,13.590221,4.061607,0.037632,0.004331,...,628.524525,1550.262019,5074.902265,0.992368,1551.894014,142.0,140.0,1514.471754,745.408996,0


### Bilanciamento del dataset

In [5]:
#separazione da tollerabile a non_tollerabile
no_guasto = df_1[df_1['Guasto'] == 0]
guasto_5 = df_1[df_1['Guasto'] == 1]
guasto_10 = df_1[df_1['Guasto'] == 2]

print("No Guasto: ", len(no_guasto), "Guasto 5%: ", len(guasto_5), "Guasto 10%: ", len(guasto_10))

No Guasto:  390 Guasto 5%:  418 Guasto 10%:  541


In [ ]:
#bilanciamento del dataset
balanced_d = pd.concat([no_guasto, guasto_5.sample(len(no_guasto)), guasto_10.sample(len(no_guasto))])
#balanced_d = pd.concat([tollerabile, non_tollerabile.sample(len(tollerabile))])
x = balanced_d.iloc[:,:-1]
y = balanced_d.iloc[:,-1:]
balanced_d.shape

In [ ]:
print(x.columns)

In [ ]:
x = df_1.iloc[:,:-1]
y = df_1.iloc[:,-1:]

In [ ]:
x = x[["<>(AccX)", "freq1(AccY)","RMS(AccX)","amp2(AccX)","var(AccX)","var(C13)","var(AccY)","var(C10)","<>(Yaw)","RMS(Yaw)","amp2(AccY)","freq1(AccZ)","var(C9)","var(C14)"]]

In [ ]:
'''oversample = SMOTE()
x, y = oversample.fit_resample(x,y)
print(x)'''

# Selezione feature più importanti

In [ ]:
from sklearn.feature_selection import SelectKBest, mutual_info_classif, f_classif, f_regression
fvalue_Best = SelectKBest(score_func=f_classif, k=5)
fit = fvalue_Best.fit(x, y)
print(fit)
mask = fvalue_Best.get_support()

print(mask)

In [ ]:
selected_features = x.columns[mask]
print("Selected features:", selected_features)

In [ ]:
features_score = pd.DataFrame(fit.scores_)
#features_score = zscore(features_score)
features = pd.DataFrame(x.columns)
feature_score = pd.concat([features,features_score],axis=1)
# Assigning column names
feature_score.columns = ["Input_Features","F_Score"]
print(feature_score.nlargest(60,columns="F_Score"))

In [ ]:
display(x['<>(AccX)'])

In [ ]:
i=0
index_false=[]
for el in list(fvalue_Best.get_support()):
    if not el:
        index_false.append(i)
    i=i+1
print(index_false)

In [ ]:
x=x.drop(x.columns[index_false],axis = 1)
print(x)

In [ ]:
plt.figure(figsize=(30,15))
ax = sns.heatmap(x[x.columns].corr(),cmap="Blues",annot=True,annot_kws={"size": 18})

In [ ]:
x = x.drop(['RMS(Yaw)'], axis=1)

In [ ]:
plt.figure(figsize=(30,15))
ax = sns.heatmap(x[x.columns].corr(),cmap="Blues",annot=True,annot_kws={"size": 18})

### Separazione del dataset in training e test

In [ ]:
# Random state
rs = 42

# Split the data to check which algorithms learn better (later on we can check )
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.8, random_state=rs)
#y_train = y_train.astype(float)
#y_test = y_test.astype(float)

# look at the shape of the data (many problems can arise from wrong shape)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
if np.array_equal(x_train, x_test) and np.array_equal(y_train, y_test):
    print("Training and testing samples are the same.")
else:
    print("Training and testing samples are different.")

In [ ]:
x_train

In [ ]:
from sklearn.calibration import CalibratedClassifierCV

# List of classifiers:
classifiers = [
    LogisticRegression(random_state = rs),
    DecisionTreeClassifier(random_state=rs, criterion="entropy"),
    RandomForestClassifier(n_estimators = 10, random_state=rs),
    GradientBoostingClassifier(random_state= rs),
    AdaBoostClassifier(n_estimators=10, random_state= rs),
    LinearDiscriminantAnalysis(),
    SVC(probability=True),
    MLPClassifier(random_state=rs), # Multi Layer Perceptron
    SGDClassifier(random_state=rs) # Stochastic Gradient Descent 
]

# List of results that will occure:
clf_name = [] # names of the classifiers
model_results = pd.DataFrame.copy(y_test) #resulting of prediction from the models

kfold = StratifiedKFold(n_splits=5) #cross-validation
cv_results = [] # scores from cross validation
cv_acc = [] # mean accuracy from cross validation, need to maximize
cv_std = [] # standard deviation from cross validation, need to minimise

cnfm = [] #confusion matrix
clr = [] #classification report
roc_auc = [] #roc curve:
roc_tpr = []
roc_fpr = []

### Classificazione

In [ ]:
clf = LogisticRegression(random_state = rs,multi_class='ovr')
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
acc = accuracy_score(y_test, y_pred)
print(acc)

In [ ]:
clf = DecisionTreeClassifier()
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
acc = accuracy_score(y_test, y_pred)
print(acc)

In [ ]:
x_train

In [ ]:
x_test

In [ ]:
from itertools import cycle
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelBinarizer

n_classes = 3

j=0
# Training the algorithms and results
for clf in classifiers:
    name = clf.__class__.__name__
    clf_name.append(name)
    
    
    #fitting and predictions
    model = clf.fit(x_train, y_train.values.ravel())
    y_pred = model.predict(x_test)
    model_results[name] = y_pred
    
    #accuracy and log loss
    cv_results.append(cross_val_score(clf, x_train, y_train.values.ravel(), scoring = "accuracy",cv = kfold)) # qui invece abbiamo cross validation
    acc = round(accuracy_score(y_test.values.ravel(), y_pred), 2) # accuratezza semplice con training e test
    train_pred = clf.predict(x_test)
    print(f'Cross-Validation: {cross_val_score(clf, x_train, y_train.values.ravel(), scoring = "accuracy",cv = kfold)}') 
    
    print(f'Accuracy: {acc} \t ---> {name} ')

    '''guasto_pred = pd.Series(model.predict(df_test), name='Guasto_test')
    conteggio = guasto_pred.value_counts()
    print(f'Test: {conteggio}')'''    
    
    #confusion matrix, clasification report, roc curve
    cnfm.append(confusion_matrix(y_test.values.ravel(), y_pred))
    clr.append(classification_report(y_test.values.ravel(), y_pred))
    #fpr, tpr, thresholds = roc_curve(y_pred, y_test.values.ravel(), pos_label=1)

    if (j == 1) | (j == 2) |(j == 7):
        y_score = clf.fit(x_train, y_train).predict_proba(x_test)
    else:
        clf = OneVsRestClassifier(clf)
        y_score = clf.fit(x_train, y_train).decision_function(x_test)

       

    lb = LabelBinarizer()
    y_true_bin = lb.fit_transform(y_test)

    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_true_bin[:, i], y_score[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
    colors = cycle(['blue', 'red', 'green'])
    for i, color in zip(range(n_classes), colors):
        plt.plot(fpr[i], tpr[i], color=color, lw=1.5,
                label='ROC curve of class {0} (area = {1:0.2f})'
                ''.format(i, roc_auc[i]))
    plt.plot([0, 1], [0, 1], 'k--', lw=1.5)
    plt.xlim([-0.05, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic for multi-class data')
    plt.legend(loc="lower right")
    plt.show()
    
    j += 1

    print("----------------------------------------------------")

for i in cv_results:
    cv_acc.append(i.mean())
    cv_std.append(i.std())


### Matrici di confusione

In [ ]:
import seaborn as sns
# Confusion matrixes (not-normalized confusion matrix)
plt.figure(figsize=(20,15))
sns.set(font_scale=1.4)
for i in range(len(classifiers)):
    plt.subplot(3,3,i+1) #adjust this acourding to the number of algorithms
    sns.heatmap(cnfm[i], annot=True, fmt="d",cmap="Reds")
    plt.subplots_adjust(hspace = 0.5)
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title(clf_name[i])

Cross Validation Scores

In [ ]:
# Cross validation accuracy results graph
cv_res = pd.DataFrame({"CrossValMeans":cv_acc, "CrossValerrors": cv_std,"Algorithm":clf_name})

plt.figure(figsize=(12,6))
sns.barplot(x="CrossValMeans",y="Algorithm",data = cv_res, palette="Set2",orient = "h",**{'xerr':cv_std})
plt.xlabel("Mean Accuracy")
plt.title("Cross validation scores")

Report Classificazione

In [ ]:
#Classification reports
for i in range(len(classifiers)):
    print (f"{clf_name[i]} Classification Report:" )
    print (clr[i])